In [1]:
%load_ext autoreload
%autoreload 2
from expressiveness_benchmark.types import Plan, Task, Language, SourceRange, Program
from code_widget.example import CodeWidget
from dataclasses import replace
import json
import pandas as pd

In [2]:
# CHANGE ME!
TASK_ID = 'inner_join'
AUTHOR = 'g'

In [34]:
task = Task(
    id=TASK_ID,
    description="Select all order ids from customers that live in California",
    plan=[
    ],
    sample_input={
       "customers":[{"cid":0,"customer":"A","location":"California"},{"cid":1,"customer":"B","location":"California"},
                    {"cid":2,"customer":"C","location":"Virginia"},{"cid":3,"customer":"D","location":"New York"}],
        "orders":[{"cid":0,"oid":4,"customer":"A","orderNum":"A1"},{"cid":0,"oid":5,"customer":"A","orderNum":"A2"},
                  {"cid":1,"oid":6,"customer":"B","orderNum":"B1"},
                 {"cid":1,"oid":7,"customer":"B","orderNum":"B2"},{"cid":2,"oid":8,"customer":"C","orderNum":"C1"}],
    },
    sample_output=[4,5,6,7],
)
task.save()

prototype = Program(
    task=TASK_ID,
    author=AUTHOR,
    language=''    
)

In [36]:
func = replace(prototype,
    language='sql',
    source='''  
    SELECT orders.oid 
    FROM 
    (orders INNER JOIN customers ON orders.cid = customers.cid) 
    WHERE location="California"
''')
func.execute(task)
func.save()

In [38]:
func = replace(prototype,
    language='datalog',
    source='''  
    inner_join(oid):-
    customers(cid,customer,location),
    location="California",
    orders(cid,_,oid,_).
''')
func.execute(task)
func.save()

In [59]:
func = replace(prototype,
    language='python-pandas',
    source='''  
def inner_join(customers,orders):
    joined = orders.join(customers, on="cid",how="inner",lsuffix="_j")
    joined = joined.loc[joined['location'] == "California"]
    return pd.DataFrame({0: list(joined.oid)})

''')
func.execute(task)
func.save()

In [61]:
func = replace(prototype,
    language='python-functional',
    source='''  
def inner_join(customers,orders):
    oids = []
    for order in orders:
        for customer in customers:
            if customer['cid'] == order['cid']:
                if customer['location'] == "California":
                    oids.append(order['oid'])
    return oids
''')
func.execute(task)
func.save()

In [ ]:
func = replace(prototype,
    language='python-imperative',
    source='''  
def inner_join(customers,orders):
    oids = []
    cids = [customer for customer in customers if customer['location'] == "California"]
    oids = [order['oid'] for order in orders if order['cid'] in c]
    for order in orders:
        for customer in customers:
            if customer['cid'] == order['cid']:
                if customer['location'] == "California":
                    oids.append(order['oid'])
    return oids
''')
func.execute(task)
func.save()